In [ ]:
%reload_ext autoreload
%autoreload 2
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
from multiprocessing import cpu_count

import numpy as np
import matplotlib.pyplot as plt
from zeus.utils import home
from monai.inferers import sliding_window_inference

from kidney.datasets.toy import create_data_loaders
from kidney.experiments.toy import ToyExperiment
from kidney.utils.checkpoints import CheckpointsStorage
from kidney.utils.image import overlay

In [ ]:
toy, checkpoint = CheckpointsStorage.create(
    ToyExperiment, home("experiments/toy/checkpoints"), "avg_val_loss"
)

In [ ]:
info = checkpoint.meta
loaders = create_data_loaders(info['data'], info['transformers'], num_workers=cpu_count())
batch = next(iter(loaders['valid']))

In [ ]:
roi_size = 96, 96
sw_batch_size = 4
img = batch["img"]
seg = batch["seg"]
out = sliding_window_inference(img, roi_size, sw_batch_size, toy.model)
mask = info['transformers'].post(outputs)
img, seg, mask = [t.squeeze().detach().numpy() for t in (img, seg, mask)]
img = np.expand_dims(img, -1)

In [ ]:
import PIL.Image

In [ ]:
raw = PIL.Image.open(batch['img_meta_dict']['filename_or_obj'][0])

In [ ]:
f, axes = plt.subplots(1, 3)
axes[0].imshow(raw, cmap='gray')
axes[1].imshow(seg, cmap='gray')
axes[2].imshow(mask, cmap='gray')

In [ ]:
# plt.imshow(overlay(img, mask, resize=None).astype(np.uint8))